In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import functional as FractionalMaxPool3d
from torch import optim

from torch.utils.data import DataLoader,Dataset, random_split

from sklearn.model_selection import train_test_split


In [2]:
TRAIN_DATA = '../data/train.csv'
TEST_DATA = '../data/test.csv'
TRAIN_IMG = '../data/train_test_data/train/'

In [3]:
train_df = pd.read_csv(TRAIN_DATA)
train_df.head()

,label,latitude,longitude,year,example_path
0,0,-2.051853,111.826093,2001,train_test_data/train/1297.png
1,2,-1.989349,105.309496,2013,train_test_data/train/1199.png
2,0,1.223256,100.702217,2014,train_test_data/train/1348.png
3,0,-2.342948,103.890226,2008,train_test_data/train/2214.png
4,0,-0.126555,101.758175,2011,train_test_data/train/2220.png


In [4]:
test_df = pd.read_csv(TEST_DATA)
test_df.head()

,latitude,longitude,year,example_path
0,0.761681,122.755954,2006,train_test_data/test/69.png
1,-8.059785,113.053791,2007,train_test_data/test/469.png
2,-2.006610,111.746316,2002,train_test_data/test/6.png
3,0.901765,114.042495,2016,train_test_data/test/351.png
4,1.911210,100.829633,2008,train_test_data/test/1001.png


In [21]:
img = Image.open(TRAIN_IMG+'2220.png')
img_array = np.asarray(img)
px.imshow(img_array)

In [22]:
img_array[0][0]

array([35, 31, 14], dtype=uint8)

In [14]:
img.size

(332, 332)

In [6]:
img.mode

'RGB'

In [35]:
def imshow(img):
  img = img / 2 + 0.5 
  npimg = img.numpy() 
  px.imshow(np.transpose(npimg, (1, 2, 0)))
  

# Custom Dataset and Dataloader

In [39]:
class ImageDataset(Dataset):
  def __init__(self,df,data_folder,transform):
    self.df = df
    self.transform = transform
    self.img_folder = data_folder
     
    self.image_names = self.df[:]['example_path']
    self.labels = self.df[:]['label']
   
  def __len__(self):
    return len(self.image_names)
 
  def __getitem__(self,index):
     
    image=Image.open(self.img_folder+self.image_names.iloc[index])

    image=self.transform(image)
    targets=self.labels[index]
     
    sample = {'image': image,'labels':targets}
 
    return sample

train_transform = transforms.Compose([
                       transforms.ToTensor(), 
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


data = ImageDataset(train_df, '../data/', train_transform)
DL = DataLoader(data, batch_size=20,shuffle=True)
sample = next(iter(DL))
print(sample['labels'][0])
img = (sample['image'][0] / 2 + 0.5 )
npimg = img.numpy() 
px.imshow(np.transpose(npimg, (1, 2, 0)))

tensor(2)


## Split Train test

In [40]:
train_data, test_data = random_split(data, [0.8, 0.2])
print(f'DATA: {len(data)}, TRAIN: {len(train_data)}, TEST: {len(test_data)}')

DATA: 1714, TRAIN: 1372, TEST: 342


In [41]:
train_DL = DataLoader(train_data, batch_size=20, shuffle=True)


## DEFINING PARAMETERS

In [105]:
from model import Net

net = Net(list(sample['image'][0].shape), classes=3)
net

Net(
  (conv): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=102400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=3, bias=True)
  )
)

In [106]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# TRAINING

In [184]:
import time
start = time.time()




for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_DL, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['image']
        labels = data['labels']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

# whatever you are timing goes here
total_time = time.time() - start

# Waits for everything to finish running
#
#torch.cuda.synchronize()

print('Finished Training')
print(total_time/60)   

Finished Training
0.021116967863506742


In [186]:
print(total_time/60)  # milliseconds



1.2670180718104045


## CALCULATE SCORE

In [189]:
test_DL = DataLoader(test_data, batch_size=20, shuffle=True)

test_sample = next(iter(test_DL))
test_sample

{'image': tensor([[[[-0.8588, -0.8667, -0.8667,  ..., -0.8902, -0.8745, -0.8588],
           [-0.8588, -0.8510, -0.8510,  ..., -0.9059, -0.8824, -0.8824],
           [-0.7333, -0.8118, -0.8196,  ..., -0.9059, -0.8824, -0.8824],
           ...,
           [-0.7725, -0.7961, -0.8196,  ..., -0.8667, -0.8824, -0.8824],
           [-0.8196, -0.7882, -0.8196,  ..., -0.8745, -0.9059, -0.8667],
           [-0.8510, -0.8510, -0.8353,  ..., -0.8824, -0.9059, -0.8980]],
 
          [[-0.7412, -0.7412, -0.7569,  ..., -0.8039, -0.7961, -0.7882],
           [-0.7333, -0.7412, -0.7412,  ..., -0.8196, -0.7961, -0.8118],
           [-0.6078, -0.7098, -0.7176,  ..., -0.8118, -0.7882, -0.8118],
           ...,
           [-0.7176, -0.7333, -0.7490,  ..., -0.8196, -0.8510, -0.8353],
           [-0.7255, -0.7176, -0.7490,  ..., -0.8510, -0.8353, -0.8118],
           [-0.7569, -0.7569, -0.7490,  ..., -0.8588, -0.8431, -0.8353]],
 
          [[-0.8824, -0.8902, -0.8980,  ..., -0.9216, -0.8980, -0.8824],
    

In [199]:


def prediction(data_loader):

    net.eval()
    y_pred = torch.LongTensor()   
    y_true = torch.LongTensor()
    for data in data_loader:
        inputs = data['image']
        y_true = torch.cat((y_true, data['labels']), dim=0)
        output = net(inputs)
      
        pred = output.cpu().data.max(1, keepdim=True)[1]
        y_pred = torch.cat((y_pred, pred), dim=0)
    
    return y_true, y_pred

y_true, y_pred = prediction(test_DL)

len(y_true)


342

In [207]:
from torchmetrics import F1Score

F1 = F1Score(num_classes=3, average='macro')
score = F1(y_pred.flatten(), y_true)
score

tensor(0.2187)

In [196]:
test_DL

In [198]:
test_data.indices

[1355,
 1001,
 276,
 773,
 1208,
 674,
 860,
 271,
 987,
 189,
 690,
 538,
 427,
 806,
 685,
 736,
 290,
 466,
 1225,
 1367,
 647,
 621,
 541,
 677,
 4,
 1190,
 1556,
 1279,
 1463,
 1317,
 292,
 1038,
 336,
 347,
 1152,
 172,
 797,
 272,
 581,
 1712,
 404,
 964,
 53,
 1616,
 1184,
 1538,
 704,
 695,
 1445,
 1045,
 1551,
 346,
 631,
 791,
 1078,
 810,
 286,
 353,
 867,
 826,
 1484,
 821,
 1640,
 1407,
 34,
 1457,
 1213,
 263,
 62,
 6,
 614,
 40,
 1092,
 1608,
 1191,
 270,
 1571,
 754,
 1063,
 665,
 858,
 321,
 856,
 141,
 1163,
 269,
 274,
 884,
 1116,
 586,
 746,
 1341,
 387,
 712,
 1211,
 73,
 65,
 1460,
 196,
 1495,
 494,
 824,
 767,
 1440,
 1271,
 1037,
 654,
 771,
 1527,
 831,
 529,
 834,
 188,
 205,
 1347,
 150,
 978,
 1693,
 945,
 1614,
 506,
 345,
 555,
 1437,
 301,
 1118,
 63,
 146,
 553,
 14,
 1141,
 1148,
 1699,
 802,
 1513,
 880,
 393,
 1618,
 1434,
 632,
 1643,
 214,
 1506,
 442,
 378,
 540,
 491,
 1381,
 1711,
 154,
 544,
 953,
 1193,
 601,
 1096,
 1219,
 1400,
 1545,
 139